<a href="https://colab.research.google.com/github/Darshan0805/Natural-Language-Processing/blob/main/Sai_Darshan_NLP6_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import random
import json
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



## Loading the intents and preprocessing it

In [20]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!',',','.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

## Converting text to numerical form to fit neural network model

In [21]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training, dtype=object)
train_x=list(training[:,0])
train_y=list(training[:,1])

## Building and fitting the model

In [22]:
model=Sequential()
model.add(Input(shape=(len(train_x[0]),)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=5, verbose=1)

Epoch 1/100
47/47 [==============================] - 1s 2ms/step - loss: 4.4006 - accuracy: 0.0086
Epoch 2/100
47/47 [==============================] - 0s 2ms/step - loss: 4.2907 - accuracy: 0.0560
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 4.1925 - accuracy: 0.0603
Epoch 4/100
47/47 [==============================] - 0s 2ms/step - loss: 4.1251 - accuracy: 0.0517
Epoch 5/100
47/47 [==============================] - 0s 2ms/step - loss: 3.9858 - accuracy: 0.0603
Epoch 6/100
47/47 [==============================] - 0s 3ms/step - loss: 3.8717 - accuracy: 0.0991
Epoch 7/100
47/47 [==============================] - 0s 2ms/step - loss: 3.7468 - accuracy: 0.0991
Epoch 8/100
47/47 [==============================] - 0s 3ms/step - loss: 3.5393 - accuracy: 0.1293
Epoch 9/100
47/47 [==============================] - 0s 3ms/step - loss: 3.4221 - accuracy: 0.1681
Epoch 10/100
47/47 [==============================] - 0s 2ms/step - loss: 3.3508 - accuracy: 0.1509
Epoch 11/

## Defining functions to perform various pre-processing tasks, predicting the class and displaying the response#

In [23]:

def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow=bag_of_words(sentence)
    res=model.predict(np.array([bow]))[0]
    error_threshold=0.25
    results=[[i,r] for i,r in enumerate(res) if r>error_threshold]
    results.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents,intents_json):
    tag=intents[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

In [26]:
while True:
    message=input("")
    ints=predict_class(message)
    res=get_response(ints,intents)
    print(res)

Hii
1/1 [==============================] - 0s 21ms/step
Hello there. Tell me how are you feeling today?
Help me
1/1 [==============================] - 0s 25ms/step
mental health is not a joke.
Thank you
1/1 [==============================] - 0s 25ms/step
Happy to help!
Ok Bye
1/1 [==============================] - 0s 22ms/step
Hi there. How are you feeling today?
Good Bye
1/1 [==============================] - 0s 19ms/step
Hi there. What brings you here today?


KeyboardInterrupt: Interrupted by user